In [1]:
import pandas as pd

In [37]:
import pandas as pd

#HAY QUE CORRER SIEMPRE DESPUES DEL DATA_LOADER.PY

# Load parquets
b3 = pd.read_parquet(r"C:\Users\Admin\Desktop\Streamlit\fco\datasets\botonera_3.parquet")
b4 = pd.read_parquet(r"C:\Users\Admin\Desktop\Streamlit\fco\datasets\botonera_4.parquet")

# Only keep needed columns
b3 = b3[["Jugadores", "Tiempo", "Fin", "Jornada"]]
b4 = b4[["Tiempo", "Fin", "Jornada"]]


# ---------------------------------------------------------------------
# 1. Merge overlapping intervals into clean contiguous blocks
# ---------------------------------------------------------------------
def merge_intervals(df):
    """Merge overlapping time intervals in a DataFrame with Inicio/Fin."""
    if df.empty:
        return []

    df_sorted = df.sort_values("Tiempo")
    merged = []
    cur_start, cur_end = df_sorted.iloc[0]["Tiempo"], df_sorted.iloc[0]["Fin"]

    for _, row in df_sorted.iloc[1:].iterrows():
        s, e = row["Tiempo"], row["Fin"]

        if s <= cur_end:  # Overlap
            cur_end = max(cur_end, e)
        else:
            merged.append((cur_start, cur_end))
            cur_start, cur_end = s, e

    merged.append((cur_start, cur_end))
    return merged


# ---------------------------------------------------------------------
# 2. Compute total intersection time between two merged interval lists
# ---------------------------------------------------------------------
def total_intersection(intervalsA, intervalsB):
    """Compute total overlap between two lists of (start,end) intervals."""
    i = j = 0
    total = 0.0

    while i < len(intervalsA) and j < len(intervalsB):
        a_start, a_end = intervalsA[i]
        b_start, b_end = intervalsB[j]

        inter_start = max(a_start, b_start)
        inter_end = min(a_end, b_end)

        if inter_end > inter_start:
            total += inter_end - inter_start

        # Move forward the interval that ends first
        if a_end < b_end:
            i += 1
        else:
            j += 1

    return total


# ---------------------------------------------------------------------
# 3. MAIN LOOP — fast evaluation for whole season
# ---------------------------------------------------------------------
results = []

for jornada in sorted(b3["Jornada"].dropna().unique()):

    b3_match = b3[b3["Jornada"] == jornada]
    b4_match = b4[b4["Jornada"] == jornada]

    if b3_match.empty or b4_match.empty:
        print(f"⚠️ Missing data for Jornada {jornada}")
        continue

    # Pre-merge active play intervals for Jornada
    active_intervals = merge_intervals(b4_match)

    # Loop players
    for player in sorted(b3_match["Jugadores"].unique()):
        player_df = b3_match[b3_match["Jugadores"] == player]

        # Merge player intervals
        player_intervals = merge_intervals(player_df)

        # Compute total effective time
        effective_seconds = total_intersection(player_intervals, active_intervals)

        results.append({
            "Jornada": jornada,
            "Jugadores": player,
            "Tiempo_Efectivo": effective_seconds
        })

df_tiempo_efectivo = pd.DataFrame(results)


# ---------------------------------------------------------------------
# 4. Save parquet
# ---------------------------------------------------------------------
output_path = r"C:\Users\Admin\Desktop\Streamlit\fco\datasets\df_tiempo_efectivo.parquet"
df_tiempo_efectivo.to_parquet(output_path, index=False)

df_tiempo_efectivo


,Jornada,Jugadores,Tiempo_Efectivo
0,1,1,1800.0
1,1,4,756.0
2,1,6,832.0
3,1,7,702.0
4,1,9,316.0
...,...,...,...
347,34,18,1019.0
348,34,19,129.0
349,34,20,17.0
350,34,21,585.0


In [41]:
df_2[df_2["Acción"] == 'Transición Defensiva']["Tipo_de_Accion"].unique()

array(['Pelota Parada', 'Repliegue', 'Tiro', 'Gol', 'Recuperacion', None,
       'Perdida'], dtype=object)

In [26]:

df_1 = pd.read_parquet(r"C:\Users\Admin\Desktop\Streamlit\fco\datasets\botonera_1.parquet")
# df_1.to_csv("df_1.csv")


df_2 = pd.read_parquet(r"C:\Users\Admin\Desktop\Streamlit\fco\datasets\botonera_2.parquet")
# df_2.to_csv("df_2.csv")


df_3 = pd.read_parquet(r"C:\Users\Admin\Desktop\Streamlit\fco\datasets\botonera_3.parquet")
# df_3.to_csv("df_3.csv")


df_4 = pd.read_parquet(r"C:\Users\Admin\Desktop\Streamlit\fco\datasets\botonera_4.parquet")
# df_4.to_csv(r"C:\Users\Admin\Desktop\Streamlit\fco\datasets\df_4_new.csv")


In [3]:
def build_effective_timeline(df_4: pd.DataFrame) -> pd.DataFrame:
    """
    Construye un timeline efectivo segundo a segundo para TODOS los partidos (Jornada),
    usando únicamente los intervalos donde el balón estuvo en juego.
    
    df_4 debe tener columnas:
        - Jornada
        - Parte
        - Inicio (segundos de vídeo donde empieza un tramo en juego)
        - Fin    (segundos de vídeo donde termina ese tramo en juego)
    """

    rows = []

    for jornada in sorted(df_4["Jornada"].unique()):
        df_j = df_4[df_4["Jornada"] == jornada]

        eff_match = 0  # contador acumulado del partido completo

        for parte in sorted(df_j["Parte"].unique()):
            df_jp = df_j[df_j["Parte"] == parte].sort_values("Tiempo")
            eff_parte = 0  # contador que se reinicia en cada parte

            for _, row in df_jp.iterrows():
                start = int(row["Tiempo"])
                end   = int(row["Fin"])

                for t in range(start, end + 1):  # segundo a segundo
                    rows.append({
                        "Jornada": jornada,
                        "Parte": parte,
                        "Tiempo_Video": t,
                        "Tiempo_Partido_Segundos": eff_match,
                    })
                    eff_match += 1
                    eff_parte += 1

    return pd.DataFrame(rows)


In [4]:
abc = build_effective_timeline(df_4)

In [5]:
abc

,Jornada,Parte,Tiempo_Video,Tiempo_Partido_Segundos
0,5,1,1,0
1,5,1,2,1
2,5,1,3,2
3,5,1,4,3
4,5,1,5,4
...,...,...,...,...
40668,32,2,6433,2558
40669,32,2,6434,2559
40670,32,2,6435,2560
40671,32,2,6436,2561


In [43]:
def map_df1_to_effective_time(df_1, timeline, max_delay=180, min_delay=30):
    """
    Maps df_1 actions (Tiempo = video timestamp) to effective match time.

    Steps:
    1. Direct match on (Jornada, Tiempo_Video)
    2. If missing → backward search (late tagging)
    3. If still missing → forward search (early tagging)
    """

    df_1 = df_1.copy()
    df_1["Tiempo_Video"] = df_1["Tiempo"].astype(int)

    # Direct merge
    merged = df_1.merge(
        timeline,
        on=["Jornada", "Tiempo_Video"],
        how="left",
        suffixes=("", "_tl")
    )

    # Missing rows
    missing_mask = merged["Tiempo_Partido_Segundos"].isna()
    if missing_mask.sum() == 0:
        return merged

    timeline_sorted = timeline.sort_values(["Jornada", "Tiempo_Video"])

    # Iterate missing rows
    for idx in merged[missing_mask].index:
        jor = merged.at[idx, "Jornada"]
        t   = merged.at[idx, "Tiempo_Video"]

        found = None

        # 1) BACKWARD search (late tag)
        for d in range(1, max_delay + 1):
            t_search = t - d
            match = timeline_sorted[
                (timeline_sorted["Jornada"] == jor) &
                (timeline_sorted["Tiempo_Video"] == t_search)
            ]
            if len(match) > 0:
                found = match.iloc[0]
                break

        # 2) FORWARD search (early tag)
        if found is None:
            for d in range(1, min_delay + 1):
                t_search = t + d
                match = timeline_sorted[
                    (timeline_sorted["Jornada"] == jor) &
                    (timeline_sorted["Tiempo_Video"] == t_search)
                ]
                if len(match) > 0:
                    found = match.iloc[0]
                    break

        # Apply if found
        if found is not None:
            for col in [
                "Parte",
                "Tiempo_Partido_Segundos",
            ]:
                merged.at[idx, col] = found[col]

    return merged


In [44]:
df_timeline = map_df1_to_effective_time(df_1, abc)
# df_timeline.to_csv(r"C:\Users\Admin\Desktop\Streamlit\fco\datasets\df_timeline.csv")

In [45]:
df_timeline

,Acción,Evento,Tiempo,Jugadores,Resultado,Control,FieldXfrom,FieldYfrom,FieldXto,FieldYto,...,GoalY,FieldX,FieldY,Sanción,Jornada,Local,Visitante,Tiempo_Video,Parte,Tiempo_Partido_Segundos
0,Tiro Ferro,Tiro Ferro 001,167,5,Desviado,mas de 2,0.764151,0.822222,0.962264,0.644444,...,NaN,NaN,NaN,None,10,Ferro,Glorias,167,1.0,151.0
1,Tiro Ferro,Tiro Ferro 002,217,10,Bloqueado,1,0.754717,0.451852,0.915094,0.511111,...,NaN,NaN,NaN,None,10,Ferro,Glorias,217,1.0,199.0
2,Tiro Ferro,Tiro Ferro 003,219,5,Atajado,1,0.858491,0.533333,0.962264,0.592593,...,NaN,NaN,NaN,None,10,Ferro,Glorias,219,1.0,201.0
3,Tiro Ferro,Tiro Ferro 004,534,5,Bloqueado,1,0.721698,0.296296,0.863208,0.400000,...,NaN,NaN,NaN,None,10,Ferro,Glorias,534,1.0,422.0
4,Tiro Ferro,Tiro Ferro 005,548,5,Bloqueado,mas de 2,0.783019,0.177778,0.886792,0.325926,...,NaN,NaN,NaN,None,10,Ferro,Glorias,548,1.0,436.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2780,Sanción Rival,Sanción Rival 003,2553,<NA>,Amarilla,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,5,Pinocho,Ferro,2553,1.0,1171.0
2781,Sanción Rival,Sanción Rival 004,2668,<NA>,Amarilla,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,5,Pinocho,Ferro,2668,1.0,1202.0
2782,Sanción Rival,Sanción Rival 005,2670,<NA>,Roja,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,5,Pinocho,Ferro,2670,1.0,1204.0
2783,Sanción Rival,Sanción Rival 006,4034,<NA>,Amarilla,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,5,Pinocho,Ferro,4034,2.0,1927.0


In [8]:
def build_minute_timeline(df_timeline: pd.DataFrame):

    df = df_timeline.copy()

    # Ensure effective time exists
    df = df[df["Tiempo_Partido_Segundos"].notna()]

    # Compute minute from seconds
    df["minuto"] = (df["Tiempo_Partido_Segundos"] // 60).astype(int)

    results = []

    # Process per match (per Jornada)
    for jornada in sorted(df["Jornada"].dropna().unique()):
        df_j = df[df["Jornada"] == jornada].copy()

        # Identify teams (should be constant)
        try:
            local = df_j["Local"].iloc[0]
            visit = df_j["Visitante"].iloc[0]
        except:
            local, visit = None, None

        # Shot indicators
        df_j["tF"] = (df_j["Acción"] == "Tiro Ferro").astype(int)
        df_j["gF"] = ((df_j["Acción"] == "Tiro Ferro") &
                      (df_j["Resultado"] == "Gol")).astype(int)

        df_j["tR"] = (df_j["Acción"] == "Tiro Rival").astype(int)
        df_j["gR"] = ((df_j["Acción"] == "Tiro Rival") &
                      (df_j["Resultado"] == "Gol")).astype(int)

        # Aggregate per minute
        df_min = (
            df_j.groupby("minuto")[["tF", "gF", "tR", "gR"]]
            .sum()
            .reset_index()
        )

        # Create full minute range (0 → last minute of match)
        max_min = int(df_j["minuto"].max())
        full = pd.DataFrame({"minuto": range(0, max_min + 1)})

        df_full = full.merge(df_min, on="minuto", how="left").fillna(0)
        df_full[["tF", "gF", "tR", "gR"]] = df_full[["tF", "gF", "tR", "gR"]].astype(int)

        # Accumulated goals
        df_full["gF_acum"] = df_full["gF"].cumsum()
        df_full["gR_acum"] = df_full["gR"].cumsum()

        # Match status (POV Ferro)
        def estado(row):
            if row["gF_acum"] > row["gR_acum"]:
                return "Ganando"
            elif row["gF_acum"] == row["gR_acum"]:
                return "Empatando"
            else:
                return "Perdiendo"

        df_full["estado"] = df_full.apply(estado, axis=1)

        # Store Jornada + Team metadata
        df_full["Jornada"] = jornada
        df_full["Local"] = local
        df_full["Visitante"] = visit

        results.append(df_full)

    # Combine all matches
    df_out = pd.concat(results, ignore_index=True)

    return df_out


In [9]:
df_minuto = build_minute_timeline(df_timeline)
df_minuto

,minuto,tF,gF,tR,gR,gF_acum,gR_acum,estado,Jornada,Local,Visitante
0,0,1,0,2,0,0,0,Empatando,5,Pinocho,Ferro
1,1,3,0,0,0,0,0,Empatando,5,Pinocho,Ferro
2,2,0,0,1,0,0,0,Empatando,5,Pinocho,Ferro
3,3,0,0,0,0,0,0,Empatando,5,Pinocho,Ferro
4,4,1,0,0,0,0,0,Empatando,5,Pinocho,Ferro
...,...,...,...,...,...,...,...,...,...,...,...
910,61,0,0,0,0,2,1,Ganando,32,Ferro,Camioneros
911,62,1,1,1,0,3,1,Ganando,32,Ferro,Camioneros
912,63,0,0,1,0,3,1,Ganando,32,Ferro,Camioneros
913,64,0,0,2,0,3,1,Ganando,32,Ferro,Camioneros


In [23]:
df_1["Acción"].unique()

array(['Tiro Ferro', 'Asistencia', '1 VS 1 Ofensivo', 'Recuperación',
       'Presión', 'Arquero', 'Tiro Rival', 'Pase Clave',
       '1 VS 1 Defensivo', 'Pérdida', 'Falta Cometida', 'Falta Recibida',
       'Sanción Ferro', 'Sanción Rival'], dtype=object)

In [22]:
df_2[df_2["Acción"] == "MT Ofensivo"]["Situacion"].unique()

array([None], dtype=object)

In [102]:
import pandas as pd
import numpy as np

def build_df2_minute_timeline(df2_eff: pd.DataFrame) -> pd.DataFrame:
    """
    Build a minute-by-minute tactical timeline from df_2 (Botonera 2),
    using effective time (Tiempo_Partido_Segundos).

    - Works for the whole season (multiple Jornadas).
    - Each row = (Jornada, Minuto) with aggregated info.
    - Inside each minute we keep events ordered chronologically.
    """

    df = df2_eff.copy()

    # ------------------------------------------------------------
    # 0. BASIC SAFETY
    # ------------------------------------------------------------
    if "Tiempo_Partido_Segundos" not in df.columns:
        raise ValueError(
            "df2_eff must contain 'Tiempo_Partido_Segundos'. "
            "Make sure you ran map_df2_to_effective_time(df_2, df_4) first."
        )

    # Drop rows without Jornada or effective time
    df = df.dropna(subset=["Jornada", "Tiempo_Partido_Segundos"])

    # Ensure numeric effective time
    df["Tiempo_Partido_Segundos"] = pd.to_numeric(
        df["Tiempo_Partido_Segundos"], errors="coerce"
    )
    df = df.dropna(subset=["Tiempo_Partido_Segundos"])

    # Convert to int seconds and minute
    df["Tiempo_Partido_Segundos"] = df["Tiempo_Partido_Segundos"].astype(int)
    df["Minuto"] = (df["Tiempo_Partido_Segundos"] // 60).astype(int)

    # Guarantee columns exist
    expected = [
        "Acción", "Tipo_de_Accion", "Resultado", "Situacion", "Duracion",
        "Jugadores", "Evento", "Local", "Visitante"
    ]
    for col in expected:
        if col not in df.columns:
            df[col] = None

    # ------------------------------------------------------------
    # 1. CLASSIFY GAME PHASE
    # ------------------------------------------------------------
    def classify_phase(a):
        if a == "MT Ofensivo":
            return "MT_OF"
        if a == "MT Defensivo":
            return "MT_DEF"
        if a == "Transición Ofensiva":
            return "T_OF"
        if a == "Transición Defensiva":
            return "T_DEF"
        return None

    df["Phase"] = df["Acción"].apply(classify_phase)

    # ------------------------------------------------------------
    # 2. CLASSIFY DURATION (Corta / Larga)
    # ------------------------------------------------------------
    def classify_duration(x):
        if pd.isna(x):
            return None
        x = str(x)
        if "Corta (H 3)" in x:
            return "Corta"
        if "Larga (+3)" in x:
            return "Larga"
        return None

    df["Duracion_Label"] = df["Duración"].apply(classify_duration)

    # ------------------------------------------------------------
    # 3. PARSE SUPERIORITIES (2 vs 1, 3 vs 2...)
    # ------------------------------------------------------------
    def parse_super(s):
        if pd.isna(s):
            return (0, 0)
        s = str(s)
        try:
            left, right = s.split("vs")
            left = int(left.strip())
            right = int(right.strip())
        except Exception:
            return (0, 0)

        if left > right:
            return (1, 0)  # For Ferro
        if right > left:
            return (0, 1)  # Against Ferro
        return (0, 0)

    df[["Sup_For", "Sup_Against"]] = df["Situacion"].apply(
        lambda x: pd.Series(parse_super(x))
    )

    # ------------------------------------------------------------
    # 4. GROUP EVENTS BY (Jornada, Evento) TO CONSOLIDATE PLAYERS
    # ------------------------------------------------------------
    # (This gives you 1 "logical event" with many players)
    event_groups = df.groupby(["Jornada", "Evento"])

    events = event_groups.apply(
        lambda g: pd.Series({
            "Minuto": g["Minuto"].min(),
            "Tiempo_Partido_Segundos": g["Tiempo_Partido_Segundos"].min(),
            "Phase": g["Phase"].iloc[0],
            "Tipo_de_Accion": g["Tipo_de_Accion"].iloc[0],
            "Resultado": g["Resultado"].iloc[0],
            "Situacion": g["Situacion"].iloc[0],
            "Duracion_Label": g["Duracion_Label"].iloc[0],
            "Sup_For": g["Sup_For"].sum(),
            "Sup_Against": g["Sup_Against"].sum(),
            "Jugadores": list(g["Jugadores"].dropna()),
            "Local": g["Local"].iloc[0],
            "Visitante": g["Visitante"].iloc[0],
        })
    ).reset_index()

    # ------------------------------------------------------------
    # 5. BUILD MINUTE-BY-MINUTE TIMELINE PER JORNADA
    # ------------------------------------------------------------
    rows = []
    group_minute = events.groupby(["Jornada", "Minuto"])

    for (jor, minute), g in group_minute:
        row = {
            "Jornada": jor,
            "Minuto": minute,
            "Local": g["Local"].iloc[0],
            "Visitante": g["Visitante"].iloc[0],
        }

        # --- PHASE COUNTS & RESULTADOS ---
        for phase in ["MT_OF", "MT_DEF", "T_OF", "T_DEF"]:
            g_phase = g[g["Phase"] == phase]

            row[f"{phase}_Total"] = len(g_phase)
            row[f"{phase}_Pos"] = (g_phase["Resultado"] == "Positivo").sum()
            row[f"{phase}_Neg"] = (g_phase["Resultado"] == "Negativo").sum()

            row[f"{phase}_Corta"] = (
                g_phase["Duracion_Label"] == "Corta"
            ).sum()
            row[f"{phase}_Larga"] = (
                g_phase["Duracion_Label"] == "Larga"
            ).sum()

            # Players involved in this phase in this minute
            players = []
            for plist in g_phase["Jugadores"]:
                players.extend(plist)
            row[f"Players_{phase}"] = players

        # --- SUPERIORITIES ---
        row["Superioridad_For"] = g["Sup_For"].sum()
        row["Superioridad_Against"] = g["Sup_Against"].sum()

        # --- CHRONOLOGICAL LIST OF EVENTS IN THIS MINUTE ---
        # Sort by effective time inside the minute
        g_sorted = g.sort_values("Tiempo_Partido_Segundos")

        # This list keeps exact order of tactical events in that minute
        row["Eventos_Ordenados"] = [
            {
                "Tiempo_Partido_Segundos": int(t),
                "Phase": ph,
                "Tipo_de_Accion": tipo,
                "Resultado": res,
                "Situacion": sit,
                "Duracion_Label": dur,
            }
            for t, ph, tipo, res, sit, dur in zip(
                g_sorted["Tiempo_Partido_Segundos"],
                g_sorted["Phase"],
                g_sorted["Tipo_de_Accion"],
                g_sorted["Resultado"],
                g_sorted["Situacion"],
                g_sorted["Duracion_Label"],
            )
        ]

        rows.append(row)

    timeline = pd.DataFrame(rows)

    # Sort nicely
    timeline = timeline.sort_values(["Jornada", "Minuto"]).reset_index(drop=True)

    return timeline


In [103]:
df_2_timeline_minuto = build_df2_minute_timeline(df_2_timeline)
df_2_timeline_minuto

C:\Users\Admin\AppData\Local\Temp\ipykernel_10704\3429333580.py:108: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  events = event_groups.apply(


,Jornada,Minuto,Local,Visitante,MT_OF_Total,MT_OF_Pos,MT_OF_Neg,MT_OF_Corta,MT_OF_Larga,Players_MT_OF,...,Players_T_OF,T_DEF_Total,T_DEF_Pos,T_DEF_Neg,T_DEF_Corta,T_DEF_Larga,Players_T_DEF,Superioridad_For,Superioridad_Against,Eventos_Ordenados
0,5,0,Pinocho,Ferro,1,1,0,0,0,"[10, 14]",...,[],1,0,0,1,0,"[5, 11, 14]",3,0,"[{'Tiempo_Partido_Segundos': 19, 'Phase': 'T_D..."
1,5,1,Pinocho,Ferro,0,0,0,0,0,[],...,"[5, 10]",0,0,0,0,0,[],2,0,"[{'Tiempo_Partido_Segundos': 88, 'Phase': 'T_O..."
2,5,2,Pinocho,Ferro,1,1,0,0,0,"[5, 10]",...,[],0,0,0,0,0,[],0,0,"[{'Tiempo_Partido_Segundos': 133, 'Phase': 'MT..."
3,5,3,Pinocho,Ferro,0,0,0,0,0,[],...,[15],0,0,0,0,0,[],0,0,"[{'Tiempo_Partido_Segundos': 223, 'Phase': 'T_..."
4,5,4,Pinocho,Ferro,0,0,0,0,0,[],...,[],1,0,0,1,0,"[4, 17]",2,0,"[{'Tiempo_Partido_Segundos': 241, 'Phase': 'MT..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,32,57,Ferro,Camioneros,0,0,0,0,0,[],...,[],0,0,0,0,0,[],0,0,"[{'Tiempo_Partido_Segundos': 3444, 'Phase': 'M..."
725,32,58,Ferro,Camioneros,0,0,0,0,0,[],...,[],0,0,0,0,0,[],0,0,"[{'Tiempo_Partido_Segundos': 3496, 'Phase': 'M..."
726,32,61,Ferro,Camioneros,1,1,0,0,0,"[8, 8, 9, 9, 14, 14, 17, 17]",...,[],0,0,0,0,0,[],0,0,"[{'Tiempo_Partido_Segundos': 3674, 'Phase': 'M..."
727,32,62,Ferro,Camioneros,0,0,0,0,0,[],...,[],0,0,0,0,0,[],0,0,"[{'Tiempo_Partido_Segundos': 3752, 'Phase': 'M..."


In [113]:
df_2_timeline_minuto.groupby("Jornada")["Minuto"].mean()

Jornada
5     21.410256
10    26.731707
12    29.000000
13    29.454545
14    19.850000
16    19.135135
19    28.612245
20    18.342857
21    30.625000
22    28.083333
25    27.108696
27    17.939394
28    28.533333
29    17.000000
30    30.836364
31    18.617647
32    31.270833
Name: Minuto, dtype: float64

In [104]:
df_2_timeline_minuto["Players_MT_OF"].value_counts()

Players_MT_OF
[]                                               275
[1, 5, 7, 10, 17]                                  9
[1, 5, 10, 11, 14]                                 9
[1, 5, 8, 10, 17]                                  9
[1, 5, 7, 10, 17, 1, 5, 7, 10, 17]                 7
                                                ... 
[2, 5, 2, 2, 10, 10]                               1
[7, 10, 6]                                         1
[7, 15]                                            1
[1, 4, 6, 7, 9, 1, 4, 6, 7, 9, 1, 4, 6, 7, 9]      1
[8, 8, 9, 9, 14, 14, 17, 17]                       1
Name: count, Length: 263, dtype: int64

In [107]:
df_2_timeline_minuto[df_2_timeline_minuto["Players_MT_OF"].isin([1, 5, 7, 10, 17, 1, 5, 7, 10, 17])]

,Jornada,Minuto,Local,Visitante,MT_OF_Total,MT_OF_Pos,MT_OF_Neg,MT_OF_Corta,MT_OF_Larga,Players_MT_OF,...,Players_T_OF,T_DEF_Total,T_DEF_Pos,T_DEF_Neg,T_DEF_Corta,T_DEF_Larga,Players_T_DEF,Superioridad_For,Superioridad_Against,Eventos_Ordenados
